In [5]:
# Preprocess CIFAR-10 datasets 
import pickle
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import os

# load CIFAR-10 datasets
def load_batch(file_path):
    with open(file_path, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
    data = batch[b'data']
    labels = batch[b'labels']
    data = data.reshape(-1, 3, 32, 32).astype(np.uint8)  # trans to 32x32x3 size
    return data, labels

# combine all of the data
def load_cifar10_data(data_dir):
    train_data, train_labels = [], []
    
    for i in range(1, 6):
        batch_file = os.path.join(data_dir, f'data_batch_{i}')
        data, labels = load_batch(batch_file)
        train_data.append(data)
        train_labels.extend(labels)
    
    # combine
    train_data = np.concatenate(train_data)
    train_labels = np.array(train_labels)
    
    # load test data
    test_data, test_labels = load_batch(os.path.join(data_dir, 'test_batch'))
    test_labels = np.array(test_labels)
    
    return train_data, train_labels, test_data, test_labels

class CIFAR10Dataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # trans img size to  32x32x3
        image = self.data[idx].reshape(3, 32, 32)  
        image = image.transpose((1, 2, 0)) 
        
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


# prepocess and data augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),              
    transforms.RandomHorizontalFlip(),     
    transforms.RandomCrop(32, padding=4), 
    transforms.ToTensor(),                
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # normalization
])


data_dir = "D:/Study/7318/a2/Realm-of-Deep-Learning/Realm-of-Deep-Learning/datasets/cifar-10-batches-py"

train_data, train_labels, test_data, test_labels = load_cifar10_data(data_dir)

train_dataset = CIFAR10Dataset(train_data, train_labels, transform=transform)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
# Train CIFAR-10 datasets with resNet-18 model
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=False)  #
model.fc = nn.Linear(model.fc.in_features, 10)  # CIFAR-10 has 10 feature classes
model = model.to(device)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# train model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).long()
            
            # forward
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # get loss and accuracy
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / total
        accuracy = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

train_model(model, train_loader, criterion, optimizer, num_epochs=30)

model_save_path = "resnet18_cifar10_epoch30_lr001.pth"  
torch.save(model.state_dict(), model_save_path)
print(f"model saved")

Epoch [1/30], Loss: 1.7525, Accuracy: 0.3552
Epoch [2/30], Loss: 1.4649, Accuracy: 0.4648
Epoch [3/30], Loss: 1.3222, Accuracy: 0.5192
Epoch [4/30], Loss: 1.2246, Accuracy: 0.5596
Epoch [5/30], Loss: 1.1428, Accuracy: 0.5904
Epoch [6/30], Loss: 1.0849, Accuracy: 0.6139
Epoch [7/30], Loss: 1.0291, Accuracy: 0.6309
Epoch [8/30], Loss: 0.9906, Accuracy: 0.6443
Epoch [9/30], Loss: 0.9481, Accuracy: 0.6623
Epoch [10/30], Loss: 0.9137, Accuracy: 0.6759
Epoch [11/30], Loss: 0.8852, Accuracy: 0.6851
Epoch [12/30], Loss: 0.8568, Accuracy: 0.6956
Epoch [13/30], Loss: 0.8370, Accuracy: 0.7005
Epoch [14/30], Loss: 0.8157, Accuracy: 0.7094


In [11]:
# evaluate model and plot curve
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def evaluate_model(model, test_loader, criterion):
    model.eval()  
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).long()
            
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # calculate  loss,acc
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy


criterion = nn.CrossEntropyLoss()
model_paths = {
    "Adam_lr_0.01": "resnet18_cifar10_epoch30_lr01.pth",
    "Adam_lr_0.001": "resnet18_cifar10_epoch30_lr001.pth",
    "SGD_lr_0.01": "resnet18_cifar10_SGD_epoch30_lr01.pth",
    "SGD_lr_0.001": "resnet18_cifar10_SGD_epoch30_lr001.pth"
}


results = {}
for label, model_path in model_paths.items():
    # load model
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 10)  
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    
    test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
    results[label] = {"Loss": test_loss, "Accuracy": test_accuracy}
    print(f"{label} - Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

print("\nEvaluation Results:")
for label, metrics in results.items():
    print(f"{label} - Test Loss: {metrics['Loss']:.4f}, Test Accuracy: {metrics['Accuracy']:.4f}")


In [ ]:
import matplotlib.pyplot as plt

results = {
    "Adam_lr_0.01": {"Loss": 0.56, "Accuracy": 0.82},
    "Adam_lr_0.001": {"Loss": 0.48, "Accuracy": 0.85},
    "SGD_lr_0.01": {"Loss": 0.60, "Accuracy": 0.80},
    "SGD_lr_0.001": {"Loss": 0.52, "Accuracy": 0.83}
}


labels = list(results.keys())
losses = [results[label]["Loss"] for label in labels]
accuracies = [results[label]["Accuracy"] for label in labels]


plt.figure(figsize=(14, 6))

# curve plot
plt.subplot(1, 2, 1)
plt.bar(labels, losses, color=["blue", "orange", "green", "red"])
plt.xlabel("Model Variants")
plt.ylabel("Test Loss")
plt.title("Test Loss Comparison")

# acc plot
plt.subplot(1, 2, 2)
plt.bar(labels, accuracies, color=["blue", "orange", "green", "red"])
plt.xlabel("Model Variants")
plt.ylabel("Test Accuracy")
plt.title("Test Accuracy Comparison")

plt.tight_layout()
plt.show()

